In [1]:
"""
This is the activity 6.2 of unit testing
Alonso Pedrero Martinez
A01769076
"""

import pandas as pd
import pathlib
import json
import os
import sys

def file_management_existence(file_catalog_input):
    """
    This function checks if the file exists.
    """
    if not pathlib.Path(file_catalog_input).is_file():
        raise FileNotFoundError("File not found: ", file_catalog_input)

    print("Found file:", file_catalog_input)

def open_file(file_name_input : str, resource : str):
    """
    Opens a file as pd dataframe
    """
    PATH = "../resources/"
    FULL_PATH = PATH + file_name_input
    file_management_existence(FULL_PATH)

    if resource == "hotel":

        df_columns = ["hotel_id", "name", "location", "rooms_total",
                        "rooms_available", "price_per_night", "rating"]
    
    elif resource == "customer":

        df_columns = ["customer_id", "name", "email", "phone"]


    try:
        df = pd.read_json(FULL_PATH)

        if df.empty == True:
            raise ValueError()

        return df
    
    except (ValueError, json.JSONDecodeError):
        print("Error: JSON file is corrupted. Using an empty DataFrame.")
        return pd.DataFrame(columns = df_columns)


def save_file(df_input : pd.DataFrame, file_name_input : str):
    PATH = "../resources/"
    FULL_PATH = PATH + file_name_input
    file_management_existence(FULL_PATH)
    df_input.to_json(FULL_PATH, orient="records", indent=4)


def parse_to(df_input : pd.DataFrame, column_name : str, data_type):
    """
    """
    df_input[column_name] = df_input[column_name].astype(data_type)


def empty_check(df_input : pd.DataFrame):
    if df_input.empty:
        print("Error: No hotels found in the database.")
        return False
    

In [2]:
class Hotel():
    """
    Basic attributes that represent the hotel
    """
    def __init__(self, hotel_id: str, name: str, location: str, rooms_total: int, price_per_night: float, rating: float):
        self.hotel_id = hotel_id
        self.name = name
        self.location = location
        self.rooms_total = rooms_total
        self.rooms_available = rooms_total
        self.price_per_night = price_per_night
        self.rating = rating

    def to_dict(self):
        """
        Convierte el hotel a un diccionario para guardarlo en JSON.
        """
        return {
            "hotel_id": self.hotel_id,
            "name": self.name,
            "location": self.location,
            "rooms_total": self.rooms_total,
            "rooms_available": self.rooms_available,
            "price_per_night": self.price_per_night,
            "rating": self.rating
        }


In [12]:
class HotelFactory():
    """
    
    """
    @staticmethod
    def create_hotel(hotel_id: str,
                     name: str,
                     location: str,
                     rooms_total: int,
                     price_per_night: float,
                     rating: float):
        """
        This method opens the hotel files and adds the 
        """
        df = open_file("hotels.json", "hotel")
        display(df)

        if hotel_id in df["hotel_id"].astype(str).values:
            print(f"Error: A hotel with ID {hotel_id} already exists.")
            return None

        hotel = Hotel(hotel_id, name, location, rooms_total, price_per_night, rating)

        df = pd.concat([df, pd.DataFrame([hotel.to_dict()])], ignore_index=True)

        save_file(df, "hotels.json")

        print(f"Hotel {name} created successfully!")
        
        return hotel
    
    @staticmethod
    def delete_hotel(hotel_id: str,
                     name: str,
                     location: str,
                     rooms_total: int,
                     price_per_night: float,
                     rating: float):
        """
        
        """
        df = open_file("hotels.json", "hotel")

        empty_check(df)
        
        parse_to(df, "hotel_id", str)
        parse_to(df, "name", str)
        parse_to(df, "location", str)
        parse_to(df, "rooms_total", int)
        parse_to(df, "price_per_night", float)
        parse_to(df, "rating", float)

        mask = (
            (df["hotel_id"] == hotel_id) &
            (df["name"] == name) &
            (df["location"] == location) &
            (df["rooms_total"] == rooms_total) &
            (df["price_per_night"] == price_per_night) &
            (df["rating"] == rating)
        )

        if df.loc[mask].empty:
            print(f"Error: No hotel found with the given attributes.")
            return False
    
        df = df.loc[~mask]

        save_file(df, "hotels.json")
        print(f"Hotel '{name}' deleted successfully.")
        
        return True


    @staticmethod
    def display_hotel_info(hotel_id : str,
                           name : str):
        """
        
        """
        df = open_file("hotels.json", "hotel")
        empty_check(df)

        parse_to(df, "hotel_id", str)
        parse_to(df, "name", str)

        df_filtered = df.loc[(df["hotel_id"] == hotel_id) & (df["name"] == name)]
        
        display(df_filtered)

    @staticmethod
    def modify_hotel_info(hotel_id: str, **kwargs):
        """
        
        """
        df = open_file("hotels.json", "hotel")
        empty_check(df)

        parse_to(df, "hotel_id", str)

        if hotel_id not in df["hotel_id"].values:
            print(f"Error: No hotel found with ID '{hotel_id}'.")
            return False

        valid_columns = ["name", "location", "rooms_total", "price_per_night", "rating"]

        for key, value in kwargs.items():
            if key in valid_columns:
                df.loc[df["hotel_id"] == hotel_id, key] = value
            else:
                print(f"Warning: '{key}' is not a valid attribute.")

        save_file(df, "hotels.json")
        print(f"Hotel '{hotel_id}' updated successfully.")
        
        return True


In [4]:
class Customer():
    """
    Basic attributes that represent the hotel
    """
    def __init__(self, customer_id: str, name: str, email: str, phone: int):
        self.customer_id = customer_id
        self.name = name
        self.email = email
        self.phone = phone

    def to_dict(self):
        """
        Convierte el hotel a un diccionario para guardarlo en JSON.
        """
        return {
            "customer_id": self.customer_id,
            "name": self.name,
            "email": self.email,
            "phone": self.phone,
        }

In [25]:
class CustomerFactory():
    """
    
    """
    @staticmethod
    def create_customer(customer_id: str,
                        name: str,
                        email: str,
                        phone: int):
        """
        This method opens the hotel files and adds the 
        """
        df = open_file("customers.json", "customer")
        
        if customer_id in df["customer_id"].astype(str).values:
            print(f"Error: A client with ID {customer_id} already exists.")
            return None

        client = Customer(customer_id, name, email, phone)


        df = pd.concat([df, pd.DataFrame([client.to_dict()])], ignore_index=True)
        

        save_file(df, "customers.json")

        print(f"Client {name} created successfully!")
        
        return client
    
    @staticmethod
    def delete_customer(customer_id: str,
                        name: str,
                        email: str,
                        phone: int):
        """
        
        """
        df = open_file("customers.json", "customer")

        empty_check(df)
        
        parse_to(df, "customer_id", str)
        parse_to(df, "name", str)
        parse_to(df, "email", str)
        parse_to(df, "phone", int)

        mask = (
            (df["customer_id"] == customer_id) &
            (df["name"] == name) &
            (df["email"] == email) &
            (df["phone"] == phone)
        )

        if df.loc[mask].empty:
            print(f"Error: No hotel found with the given attributes.")
            return False
    
        df = df.loc[~mask]

        save_file(df, "customers.json")
        print(f"customer '{name}' deleted successfully.")
        
        return True


    @staticmethod
    def display_customer_info(customer_id : str,
                           name : str):
        """
        
        """
        df = open_file("customers.json", "customer")
        empty_check(df)

        parse_to(df, "customer_id", str)
        parse_to(df, "name", str)

        df_filtered = df.loc[(df["customer_id"] == customer_id) & (df["name"] == name)]
        
        display(df_filtered)

    @staticmethod
    def modify_customer_info(customer_id: str, **kwargs):
        """
        
        """
        df = open_file("customers.json", "customer")
        empty_check(df)

        parse_to(df, "customer_id", str)

        if customer_id not in df["customer_id"].values:
            print(f"Error: No customer found with ID '{customer_id}'.")
            return False

        valid_columns = ["name", "email", "phone"]

        for key, value in kwargs.items():
            if key in valid_columns:
                df.loc[df["customer_id"] == customer_id, key] = value
            else:
                print(f"Warning: '{key}' is not a valid attribute.")

        save_file(df, "customers.json")
        print(f"Customer '{customer_id}' updated successfully.")
        
        return True


In [6]:
HotelFactory.create_hotel("1", "Holiday Inn", "MX", 30, 305.0, 5.0)

Found file: ../resources/hotels.json
Error: JSON file is corrupted. Using an empty DataFrame.


,hotel_id,name,location,rooms_total,rooms_available,price_per_night,rating


Found file: ../resources/hotels.json
Hotel Holiday Inn created successfully!


/var/folders/xj/856h4cfd3k52z_02341c_sxc0000gn/T/ipykernel_14694/694850704.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([hotel.to_dict()])], ignore_index=True)


In [7]:
HotelFactory.display_hotel_info("1", "Holiday Inn")

Found file: ../resources/hotels.json


,hotel_id,name,location,rooms_total,rooms_available,price_per_night,rating
0,1,Holiday Inn,MX,30,30,305,5


In [8]:
HotelFactory.delete_hotel("1", "Holiday Inn", "MX", 30, 305.0, 5.0)

Found file: ../resources/hotels.json
Found file: ../resources/hotels.json
Hotel 'Holiday Inn' deleted successfully.


True

In [9]:
HotelFactory.create_hotel("1", "Holiday Inn", "MX", 30, 305.0, 5.0)

Found file: ../resources/hotels.json
Error: JSON file is corrupted. Using an empty DataFrame.


,hotel_id,name,location,rooms_total,rooms_available,price_per_night,rating


Found file: ../resources/hotels.json
Hotel Holiday Inn created successfully!


/var/folders/xj/856h4cfd3k52z_02341c_sxc0000gn/T/ipykernel_14694/694850704.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([hotel.to_dict()])], ignore_index=True)


In [10]:
HotelFactory.display_hotel_info("1", "Holiday Inn")

Found file: ../resources/hotels.json


,hotel_id,name,location,rooms_total,rooms_available,price_per_night,rating
0,1,Holiday Inn,MX,30,30,305,5


In [13]:
HotelFactory.modify_hotel_info("1", name="New ONE", price_per_night=350.0)

Found file: ../resources/hotels.json
Found file: ../resources/hotels.json
Hotel '1' updated successfully.


True

In [79]:
HotelFactory.display_hotel_info("2", "New ONE")

Found file: ../resources/hotels.json


,hotel_id,name,location,rooms_total,rooms_available,price_per_night,rating
0,2,New ONE,MX,30,30,350,5


In [19]:
CustomerFactory.create_customer("1", "John", "a@gmail.com", 722)

Found file: ../resources/customers.json
Error: JSON file is corrupted. Using an empty DataFrame.


,customer_id,name,email,phone


Found file: ../resources/customers.json
Client John created successfully!


In [20]:
CustomerFactory.display_customer_info("1", "John")

Found file: ../resources/customers.json


,customer_id,name,email,phone
0,1,John,a@gmail.com,722


In [26]:
CustomerFactory.modify_customer_info("2", name = "Juan")

Found file: ../resources/customers.json
Error: No customer found with ID '2'.


False

In [27]:
CustomerFactory.delete_customer("1", "Juan", "a@gmail.com", 722)

Found file: ../resources/customers.json
Found file: ../resources/customers.json
customer 'Juan' deleted successfully.


True